In [3]:
def findgiftis(FOLDER, ext):

    # Function goal: return nifti files within a directory

    # Detail: could be used for a file with any extension...

    # Inputs

    # FOLDER (directory to search)
    # ext (the suffix (.gii or .gii.gz))

    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)
    import os
    import sys
    giftilist = list()
    for file in os.listdir(FOLDER):
        if not file.startswith('.') and file.endswith(ext):
            giftilist.append(os.path.join(FOLDER, file))
    fprintf(sys.stdout, "Found %d gifti files \n", len(giftilist))
    return(sorted(giftilist))

In [4]:
def make_gifti_list(base_folder, sessionstring, space):

    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)

    import os
    import sys
    sessionlist = list()
    for session in os.listdir(base_folder):
        if not session.startswith('.') and session.startswith(tuple(sessionstring)):
            sessionlist.append(os.path.join(base_folder, session))
    fprintf(sys.stdout, "Found %d sessions \n", len(sessionlist))

    gifts = []

    for sess in sorted(sessionlist):
        gifts.append(findgiftis(os.path.join(sess, 'func'), '.gii'))

    flatgifts = []

    for sublist in gifts:
        for item in sublist:
            flatgifts.append(item)

    leftgifts = [gift for gift in flatgifts if space + "_hemi-L" in(gift)]
    rightgifts = [gift for gift in flatgifts if space + "_hemi-R" in(gift)]

    return leftgifts, rightgifts, gifts, flatgifts

In [5]:
mygifts=make_gifti_list('/Volumes/BAHAMUT/ALL_PRF_DAT/BIGRUN/derivatives/fmriprep/sub-02','ses','fsaverage')

Found 4 sessions 
Found 52 gifti files 
Found 32 gifti files 
Found 40 gifti files 
Found 60 gifti files 


In [7]:
from nilearn import surface

from joblib import Parallel, delayed

In [13]:
x=surface.load_surf_data(mygifts[0][0]).astype('float16')
type(x)

numpy.ndarray

In [14]:
x[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float16)

In [15]:
from nilearn import surface

from joblib import Parallel, delayed

In [26]:
def import_surf_astype(fn,fmt):
    impt=surface.load_surf_data(fn).astype(fmt)
    return impt

In [28]:
def import_gifts(flist, procs,fmt):
    gifts = Parallel(n_jobs=procs, verbose=9)(delayed(import_surf_astype)(fn,fmt) for fn in flist)
    return gifts

In [29]:
Lgifts=import_gifts(mygifts[0],5,'float32')

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    9.7s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   29.6s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:   55.8s remaining:    3.9s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:   58.3s finished


In [43]:
type(Lgifts[45][0,0])

numpy.float32